In [65]:
# !source activate notebook
# !conda install -c fastai -c pytorch -y fastai
# !conda install -y voila
# !pip install -Uqq  fastbook

In [66]:
import os
import torch
import torchvision
import numpy
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from fastbook import *
from fastai.vision.widgets import *
from network import U_Net
from IPython.display import clear_output, Image , display
# !jupyter serverextension enable --sys-prefix voila
clear_output()

    

这是一个基于UNet，用于皮肤病影像图像分割的实现。
上传图片会自动输出并保存结果。

In [67]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()

In [68]:
unet = U_Net(img_ch=3,output_ch=1)
unet.train(False)
unet.eval()

state_dict = torch.load('/home/jovyan/U_Net-4-16.pth', map_location='cpu')  
# state_dict = torch.load('.\\U_Net-4-16.pth', map_location='cpu')  #for PC
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`，表面从第7个key值字符取到最后一个字符，正好去掉了module.
    new_state_dict[name] = v #新字典的key值对应的value为一一对应的值。 
# load params
unet.load_state_dict(new_state_dict)
clear_output()

In [69]:
Transform = []
Transform.append(transforms.Resize((224,224)))
Transform.append(transforms.ToTensor())
Transform = transforms.Compose(Transform)

In [70]:
def on_data_change(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    img_new = img
    img_new = Transform(img_new)
    img_new = img_new.unsqueeze(0)
    out = unet(img_new)
    torchvision.utils.save_image(out, 'test.png')
    image = Image('test.png')
    with out_pl:  display(image)

In [71]:
# def on_data_change(change):
#     img = PILImage.create(btn_upload.data[-1])
#     out_pl.clear_output()
#     img_new = img
#     img_new = Transform(img_new)
#     img_new = img_new.unsqueeze(0)
#     out = unet(img_new)
#     torchvision.utils.save_image(out, 'test.png')
#     out = out.detach().numpy()
#     out = out.squeeze()
#     plt.imshow(out,cmap = plt.cm.gray)
#     plt.axis('off')
#     with out_pl:  display(img.resize((200,200))) 
#     with out_pl:  plt.show()

In [72]:
# def on_data_change(change):
#     img = PILImage.create(btn_upload.data[-1])
#     out_pl.clear_output()
#     with out_pl: display(img)
#     img = Transform(img)
#     img = img.unsqueeze(0)
#     out = unet(img)
#     torchvision.utils.save_image(out, 'test.png')
#     out = out.squeeze(0)
#     out = transforms.ToPILImage()(out)
#     with out_pl: display(out)
        

In [73]:
btn_upload.observe(on_data_change, names=['data'])

In [74]:
display(VBox([widgets.Label('请上传图片'), btn_upload, out_pl]))